In [9]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, model_validator
from trustcall import create_extractor

model = init_chat_model("openai:gpt-4o-mini")


def improve_prompt(prompt: ChatPromptTemplate):
    variables = prompt.input_variables

    class ImprovePrompt(BaseModel):
        thought: str
        improved: str

        @model_validator(mode="after")
        def validate_improved(cls, self):
            new_variables = ChatPromptTemplate.from_template(
                self.improved
            ).input_variables
            assert set(new_variables) == set(variables), (
                "Improved prompt must contain the same fstring variables as the input."
                "\nExpected: {variables}.\nGot {new_variables}."
            )
            return self

    extractor = create_extractor(
        model, tools=[ImprovePrompt], tool_choice="ImprovePrompt"
    )
    result = extractor.invoke(
        "Improve this prompt a lot. Feel free to improve the naming of the fstring args"
        + str(prompt.messages[0].prompt.template)
    )
    return result


prompt = ChatPromptTemplate.from_template(
    "You are a {foo} assistant. Respond with {bar}"
)

improve_prompt(prompt)